In [49]:
import pandas as pd
import os
from datetime import timedelta, date, datetime
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR, DATE, FLOAT, VARCHAR, DATETIME

import config

data_dir = "data"

engine = create_engine(
    f"mysql+pymysql://{config.sql_credentials['user']}:{config.sql_credentials['password']}@{config.sql_credentials['host']}:{config.sql_credentials['port']}/{config.sql_credentials['db']}",
    pool_pre_ping=True)

conn = engine.connect()

In [35]:
et_query = '''
            SELECT placement_ad_id, carer_id, sent_at, sms_type
            FROM live_STATS_MATCHING_MATCH_REQUEST_CARER_SMS
            WHERE placement_ad_id IS NOT NULL AND
                  placement_id IS NOT NULL AND
                  sent_at>'2019-08-01' and
                  sms_type in ('urgentMatchSent','slowMatchingSent','preferredPlacementSent','initialPromotionSent','returningCarerSent')
         '''
et_sms = pd.read_sql(et_query, conn)

In [36]:
et_sms

,placement_ad_id,carer_id,sent_at,sms_type
0,plad_pl_s0DyQnvG3lBw_tnCT-b8d6476a-d89e-4952-9...,p100032767,2020-09-29 14:30:28,initialPromotionSent
1,plad_pl_s03ixZ5n2boc_mNFS-7805b74d-2327-4741-a...,p100037528,2020-08-20 10:59:55,initialPromotionSent
2,plad_pl_s0dDY73S0c5C_jWip-1b430c7e-cc09-49ec-8...,p100037528,2020-08-04 13:05:12,initialPromotionSent
3,plad_pl_s0HURQfyMqyw_baqe-42e9dcc6-ec29-493e-9...,p100037528,2020-08-18 15:44:45,initialPromotionSent
4,plad_pl_s0hxSYNLPFdB_CmNi-d0c5f403-ff92-4354-8...,p100037528,2020-07-02 13:59:48,initialPromotionSent
...,...,...,...,...
835107,plad_pl_s0yteJl8rKlb_aEkV-67966ad1-acc7-4f8a-a...,p99950970,2021-05-17 13:41:14,urgentMatchSent
835108,plad_pl_s0yvV8v1z4gj_qhlV-9ed7bef5-c52c-46f6-b...,p99950970,2020-08-23 07:11:40,urgentMatchSent
835109,plad_pl_s0YYpC0B0mBS_7UOM-86e7f86d-4fb3-4d3b-8...,p99950970,2021-04-24 08:26:03,urgentMatchSent
835110,plad_pl_s0ZaRT3oAt8K_8RJa-c7f14e28-ed62-4c41-a...,p99950970,2020-09-25 07:11:35,urgentMatchSent


In [41]:
# import csvs

data = pd.DataFrame()

df_names = ['carer_df_1909_2001',
            'carer_df_2001_2002',
            'carer_df_2002_2006',
            'carer_df_2006_2101',
            'carer_df_2101_2103',
            'carer_df_2103_2108']

for f in df_names:

    filename = f'../data/bulk_messages/output_files/{f}.csv'
    if os.path.isfile(filename):
        print(f'{f} is being appended')

        carer_df = pd.read_csv(filename, parse_dates=['sent_at'])
        data = data.append(carer_df)

print(data.shape)
data.drop_duplicates(subset=['placement_ad_id','carer_id','sent_at'],inplace = True) # more than one number per carer?
print(data.shape)


carer_df_1909_2001 is being appended
carer_df_2001_2002 is being appended
carer_df_2002_2006 is being appended
carer_df_2006_2101 is being appended
carer_df_2101_2103 is being appended
carer_df_2103_2108 is being appended
(1232843, 4)
(1174932, 4)


In [42]:
print(data.sent_at.min(),data.sent_at.max())

2019-09-17 13:12:00 2021-08-17 18:11:00


In [43]:
data['sms_type'] = 'customListSent'

data['placement_ad_id_carer_id_sent_at'] = data['placement_ad_id'] + data['carer_id'] + data['sent_at'].apply(lambda x:str(x))
print(data['placement_ad_id_carer_id_sent_at'].nunique()==data.shape[0])

True


In [46]:

dtypes = {
    'sent_at': DATETIME,
}

data_columns = [
                'carer_id',
                'placement_ad_id',
                'sent_at',
                'sms_type',
                'placement_ad_id_carer_id_sent_at'
                ]

data = data[data_columns]

data.head()

,carer_id,placement_ad_id,sent_at,sms_type,placement_ad_id_carer_id_sent_at
0,p258326082,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...,2019-09-17 13:12:00,customListSent,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...
1,p91963657,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...,2019-09-17 13:12:00,customListSent,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...
2,p54540830,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...,2019-09-17 13:12:00,customListSent,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...
3,p73830676,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...,2019-09-17 13:12:00,customListSent,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...
4,p96258457,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...,2019-09-17 13:12:00,customListSent,plad_pl_s03iuz2MBGtu_UJ2Y-7f6e6167-c776-447a-a...


In [50]:
print(f'Starting writing table at {datetime.now()}')

data[data_columns].sort_values(['sent_at','placement_ad_id']).to_sql(
    'live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS',
    con=conn,
    schema="elder_live",
    if_exists="replace",
    index=False,
    dtype= dtypes
)

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` CHANGE COLUMN `placement_ad_id` `placement_ad_id` VARCHAR(128) NOT NULL;')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` CHANGE COLUMN `sent_at` `sent_at` DATETIME NOT NULL;')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` CHANGE COLUMN `carer_id` `carer_id` VARCHAR(16) NOT NULL;')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` CHANGE COLUMN `sms_type` `sms_type` VARCHAR(32) NOT NULL;')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` ADD UNIQUE INDEX `placement_ad_id_sent_at_carer_id` (`placement_ad_id` ASC, `sent_at` ASC, `carer_id` ASC);')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` ADD UNIQUE INDEX `sent_at_placement_ad_id_carer_id` (`sent_at` ASC,`placement_ad_id` ASC, `carer_id` ASC);')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` ADD UNIQUE INDEX `carer_id_sent_at_placement_ad_id` (`carer_id` ASC, `sent_at` ASC,`placement_ad_id` ASC);')

conn.execute(
        'ALTER TABLE `elder_live`.`live_MO_CUSTOM_LIST_MATCH_REQUEST_CARER_SMS` ADD UNIQUE INDEX `sent_at_carer_id_placement_ad_id` (`sent_at` ASC,`carer_id` ASC,`placement_ad_id` ASC);')


print(f'Completing writing table at {datetime.now()}')



Starting writing table at 2021-09-03 09:23:49.531803
Completing writing table at 2021-09-03 09:28:53.306041
